<h1>Spreadsheet processing</h1> <br /><br>
<p>This example covers<br>
    <ul><br>
        <li>Extracting data from spreadsheets;</li><br>
        <li>Re-structuring data in a desired format;</li><br>
        <li>Saving re-structured data into a relational database.</li><br>
    </ul><br>
</p> <br />